In [ ]:
# default_exp data.question_answering

In [ ]:
#all_slow

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# data.question_answering

> This module contains the bits required to use the fastai DataBlock API and/or mid-level data processing pipelines to organize your data for question/answering tasks.

In [ ]:
#export
import ast
from functools import reduce

from blurr.utils import *
from blurr.data.core import *

import torch
from transformers import *
from fastai.text.all import *

logging.set_verbosity_error()

In [ ]:
#hide_input
import pdb

from nbdev.showdoc import *
from fastcore.test import *

from fastai import __version__ as fa_version
from torch import __version__ as pt_version
from transformers import __version__ as hft_version

print(f'Using pytorch {pt_version}')
print(f'Using fastai {fa_version}')
print(f'Using transformers {hft_version}')

Using pytorch 1.7.1
Using fastai 2.4
Using transformers 4.8.1


In [ ]:
#hide_input
#cuda
torch.cuda.set_device(1)
print(f'Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}')

Using GPU #1: GeForce GTX 1080 Ti


## Question/Answering tokenization, batch transform, and DataBlock methods

Question/Answering tasks are models that require two text inputs (a context that includes the answer and the question).  The objective is to predict the start/end tokens of the answer in the context)

In [ ]:
path = Path('./')
squad_df = pd.read_csv(path/'squad_sample.csv'); len(squad_df)

1000

We've provided a simple subset of a pre-processed SQUADv2 dataset below just for demonstration purposes. There is a lot that can be done to make this much better and more fully functional.  The idea here is just to show you how things can work for tasks beyond sequence classification. 

In [ ]:
squad_df.head(2)

,id,title,context,question,answers,ds_type,answer_text,is_impossible
0,56be85543aeaaa14008c9063,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five G...",When did Beyonce start becoming popular?,"{'text': ['in the late 1990s'], 'answer_start': [269]}",train,in the late 1990s,False
1,56be85543aeaaa14008c9065,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five G...",What areas did Beyonce compete in when she was growing up?,"{'text': ['singing and dancing'], 'answer_start': [207]}",train,singing and dancing,False


In [ ]:
model_cls = AutoModelForQuestionAnswering

pretrained_model_name = 'roberta-base' #'xlm-mlm-ende-1024'
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR.get_hf_objects(pretrained_model_name, model_cls=model_cls)

In [ ]:
#export
def pre_process_squad(row, hf_arch, hf_tokenizer, ctx_attr='context', qst_attr='question', ans_attr='answer_text'):
    context, qst, ans = row[ctx_attr], row[qst_attr], row[ans_attr]
    
    tok_kwargs = {}

    if(hf_tokenizer.padding_side == 'right'):
        tok_input = hf_tokenizer.convert_ids_to_tokens(hf_tokenizer.encode(qst, context, **tok_kwargs))
    else:
        tok_input = hf_tokenizer.convert_ids_to_tokens(hf_tokenizer.encode(context, qst, **tok_kwargs))
                                                                       
    tok_ans = hf_tokenizer.tokenize(str(row[ans_attr]), **tok_kwargs)
    
    start_idx, end_idx = 0,0
    for idx, tok in enumerate(tok_input):
        try:
            if (tok == tok_ans[0] and tok_input[idx:idx + len(tok_ans)] == tok_ans): 
                start_idx, end_idx = idx, idx + len(tok_ans)
                break
        except: pass
            
    row['tokenized_input'] = tok_input
    row['tokenized_input_len'] = len(tok_input)
    row['tok_answer_start'] = start_idx
    row['tok_answer_end'] = end_idx
    
    return row

The `pre_process_squad` method is structured around how we've setup the squad DataFrame above.

In [ ]:
squad_df = squad_df.apply(partial(pre_process_squad, hf_arch=hf_arch, hf_tokenizer=hf_tokenizer), axis=1)

In [ ]:
max_seq_len= 128

In [ ]:
squad_df = squad_df[(squad_df.tok_answer_end < max_seq_len) & (squad_df.is_impossible == False)]

In [ ]:
#hide
squad_df.head(2)

,id,title,context,question,answers,ds_type,answer_text,is_impossible,tokenized_input,tokenized_input_len,tok_answer_start,tok_answer_end
0,56be85543aeaaa14008c9063,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five G...",When did Beyonce start becoming popular?,"{'text': ['in the late 1990s'], 'answer_start': [269]}",train,in the late 1990s,False,"[<s>, ĠWhen, Ġdid, ĠBeyon, ce, Ġstart, Ġbecoming, Ġpopular, ?, </s>, </s>, ĠBeyon, cÃ©, ĠG, is, elle, ĠKnow, les, -, Carter, Ġ(/, bi, Ë, Ĳ, ËĪ, j, É, Ĵ, n, se, É, ª, /, Ġbee, -, Y, ON, -, say, ), Ġ(, born, ĠSeptember, Ġ4, ,, Ġ1981, ), Ġis, Ġan, ĠAmerican, Ġsinger, ,, Ġsong, writer, ,, Ġrecord, Ġproducer, Ġand, Ġactress, ., ĠBorn, Ġand, Ġraised, Ġin, ĠHouston, ,, ĠTexas, ,, Ġshe, Ġperformed, Ġin, Ġvarious, Ġsinging, Ġand, Ġdancing, Ġcompetitions, Ġas, Ġa, Ġchild, ,, Ġand, Ġrose, Ġto, Ġfame, Ġin, Ġthe, Ġlate, Ġ1990, s, Ġas, Ġlead, Ġsinger, Ġof, ĠR, &, B, Ġgirl, -, group, ĠDestiny, ...]",185,84,89
1,56be85543aeaaa14008c9065,Beyoncé,"Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five G...",What areas did Beyonce compete in when she was growing up?,"{'text': ['singing and dancing'], 'answer_start': [207]}",train,singing and dancing,False,"[<s>, ĠWhat, Ġareas, Ġdid, ĠBeyon, ce, Ġcompete, Ġin, Ġwhen, Ġshe, Ġwas, Ġgrowing, Ġup, ?, </s>, </s>, ĠBeyon, cÃ©, ĠG, is, elle, ĠKnow, les, -, Carter, Ġ(/, bi, Ë, Ĳ, ËĪ, j, É, Ĵ, n, se, É, ª, /, Ġbee, -, Y, ON, -, say, ), Ġ(, born, ĠSeptember, Ġ4, ,, Ġ1981, ), Ġis, Ġan, ĠAmerican, Ġsinger, ,, Ġsong, writer, ,, Ġrecord, Ġproducer, Ġand, Ġactress, ., ĠBorn, Ġand, Ġraised, Ġin, ĠHouston, ,, ĠTexas, ,, Ġshe, Ġperformed, Ġin, Ġvarious, Ġsinging, Ġand, Ġdancing, Ġcompetitions, Ġas, Ġa, Ġchild, ,, Ġand, Ġrose, Ġto, Ġfame, Ġin, Ġthe, Ġlate, Ġ1990, s, Ġas, Ġlead, Ġsinger, Ġof, ĠR, &, ...]",190,77,80


In [ ]:
vocab = dict(enumerate(range(max_seq_len)))

In [ ]:
#export
class HF_QuestionAnswerInput(HF_BaseInput): pass

We'll return a `HF_QuestionAnswerInput` from our custom `HF_BeforeBatchTransform` so that we can customize the show_batch/results methods for this task.

In [ ]:
#export
class HF_QABeforeBatchTransform(HF_BeforeBatchTransform):
    def __init__(self, hf_arch, hf_config, hf_tokenizer, hf_model,
                 max_length=None, padding=True, truncation=True,  is_split_into_words=False, 
                 tok_kwargs={}, **kwargs):
                 
        super().__init__(hf_arch, hf_config, hf_tokenizer, hf_model,
                         max_length=max_length, padding=padding, truncation=truncation, 
                         is_split_into_words=is_split_into_words, tok_kwargs=tok_kwargs, **kwargs)
        
    def encodes(self, samples):  
        samples = super().encodes(samples)
        for s in samples:
            # cls_index: location of CLS token (used by xlnet and xlm); is a list.index(value) for pytorch tensor's
            s[0]['cls_index'] = (s[0]['input_ids'] == self.hf_tokenizer.cls_token_id).nonzero()[0]
            # p_mask: mask with 1 for token than cannot be in the answer, else 0 (used by xlnet and xlm)
            s[0]['p_mask'] = s[0]['special_tokens_mask']
            
        return samples

By overriding `HF_BeforeBatchTransform` we can add other inputs to each example for this particular task.

In [ ]:
before_batch_tfm = HF_QABeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model,
                                             max_length=max_seq_len, truncation='only_second', 
                                             tok_kwargs={ 'return_special_tokens_mask': True })

blocks = (
    HF_TextBlock(before_batch_tfm=before_batch_tfm, input_return_type=HF_QuestionAnswerInput), 
    CategoryBlock(vocab=vocab),
    CategoryBlock(vocab=vocab)
)

dblock = DataBlock(blocks=blocks, 
                   get_x=lambda x: (x.question, x.context),
                   get_y=[ColReader('tok_answer_start'), ColReader('tok_answer_end')],
                   splitter=RandomSplitter(),
                   n_inp=1)

In [ ]:
dls = dblock.dataloaders(squad_df, bs=4)

In [ ]:
b = dls.one_batch(); len(b), len(b[0]), len(b[1]), len(b[2])

(3, 5, 4, 4)

In [ ]:
b[0]['input_ids'].shape, b[0]['attention_mask'].shape, b[1].shape, b[2].shape

(torch.Size([4, 128]), torch.Size([4, 128]), torch.Size([4]), torch.Size([4]))

In [ ]:
#export
@typedispatch
def show_batch(x:HF_QuestionAnswerInput, y, samples, dataloaders, ctxs=None, max_n=6, trunc_at=None, **kwargs):  
    # grab our tokenizer
    hf_before_batch_tfm = get_blurr_tfm(dataloaders.before_batch)
    hf_tokenizer = hf_before_batch_tfm.hf_tokenizer
    
    res = L()
    for sample, input_ids, start, end in zip(samples, x, *y):
        txt = hf_tokenizer.decode(sample[0], skip_special_tokens=True)[:trunc_at]

        ans_toks = hf_tokenizer.convert_ids_to_tokens(input_ids, skip_special_tokens=False)[start:end]
        res.append((txt, (start.item(),end.item()), hf_tokenizer.convert_tokens_to_string(ans_toks)))
                       
    display_df(pd.DataFrame(res, columns=['text', 'start/end', 'answer'])[:max_n])
    return ctxs

The `show_batch` method above allows us to create a more interpretable view of our question/answer data.

In [ ]:
dls.show_batch(dataloaders=dls, max_n=2, trunc_at=500)

,text,start/end,answer
0,"What was the child's name? On January 7, 2012, Beyoncé gave birth to her first child, a daughter, Blue Ivy Carter, at Lenox Hill Hospital in New York. Five months later, she performed for four nights at Revel Atlantic City's Ovation Hall to celebrate the resort's opening, her first performances since giving birth to Blue Ivy.","(28, 31)",Blue Ivy Carter
1,"Which of Beyonce's songs was called Greatest Song of the 2000s? Her debut single, ""Crazy in Love"" was named VH1's ""Greatest Song of the 2000s"", NME's ""Best Track of the 00s"" and ""Pop Song of the Century"", considered by Rolling Stone to be one of the 500 greatest songs of all time, earned two Grammy Awards and is one of the best-selling singles of all time at around 8 million copies. The music video for ""Single Ladies (Put a Ring on It)"", which achieved fame for its intricate choreography and it","(0, 0)",


## Summary

This module includes all the low, mid, and high-level API bits for extractive Q&A tasks data preparation.

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 01_data-core.ipynb.
Converted 01_modeling-core.ipynb.
Converted 02_data-language-modeling.ipynb.
Converted 02_modeling-language-modeling.ipynb.
Converted 03_data-token-classification.ipynb.
Converted 03_modeling-token-classification.ipynb.
Converted 04_data-question-answering.ipynb.
Converted 04_modeling-question-answering.ipynb.
Converted 10_data-seq2seq-core.ipynb.
Converted 10_modeling-seq2seq-core.ipynb.
Converted 11_data-seq2seq-summarization.ipynb.
Converted 11_modeling-seq2seq-summarization.ipynb.
Converted 12_data-seq2seq-translation.ipynb.
Converted 12_modeling-seq2seq-translation.ipynb.
Converted 99a_examples-high-level-api.ipynb.
Converted 99b_examples-glue.ipynb.
Converted 99c_examples-glue-plain-pytorch.ipynb.
Converted 99d_examples-multilabel.ipynb.
Converted index.ipynb.
